In [ ]:
import numpy
import cv2
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
s=cv2.imread('/mnt/Personal/Projects/Python/Autofocus/test.exr',  cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)  
print(type(s))
print(numpy.percentile(s,84.44357))

In [ ]:
# first import os and enable the necessary flags to avoid cv2 errors

import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
import cv2

# then just type in following



img=cv2.imread('/mnt/Velocity Vault/test.exr',  cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)  
'''
you might have to disable following flags, if you are reading a semantic map/label then because it will convert it into binary map so check both lines and see what you need
''' 
# img = cv2.imread(PATH2EXR) 
 
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
import os

def display_txt_files_contents(directory):
    file_count = 0
    try:
        # Walk through the directory and all subdirectories
        for root, dirs, files in os.walk(directory):
            # Filter out .txt files
            txt_files = [f for f in files if f.endswith('.txt')]
            
            # Read and display the contents of each .txt file
            for txt_file in txt_files:
                file_path = os.path.join(root, txt_file)
                with open(file_path, 'r') as file:
                    print(f"Contents of {file_path}:\n")
                    print(file.read())
                    print("\n" + "-"*40 + "\n")
                file_count += 1
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        print(f"Number of .txt files processed: {file_count}")

# Example usage
directory_path = '/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/Train'
display_txt_files_contents(directory_path)


In [ ]:
import os

def parse_file_content(file_path):
    data = {
        'pos': [],
        'orien': [],
        'f': None,
        'par': None,
        'pp': [],
        'rd': [],
        'skew': None,
        'size': [],
        'pt': None
    }
    
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split(': ')
            if key == 'position':
                data['pos'].append(float(value))
            elif key == 'orientation':
                data['orien'].append(float(value))
            elif key == 'focal_length':
                data['f'] = float(value)
            elif key == 'pixel_aspect_ratio':
                data['par'] = int(value)
            elif key == 'principal_point':
                data['pp'].append(float(value))
            elif key == 'radial_distortion':
                data['rd'].append(float(value))
            elif key == 'skew':
                data['skew'] = int(value)
            elif key == 'size_x':
                data['size'].append(int(value))
            elif key == 'size_y':
                data['size'].append(int(value))
            elif key == 'projection_type':
                data['pt'] = value

    return data

def create_scale_dictionary(directory):
    scale = {}
    txt_file_count = 0

    try:
        # Walk through the directory and all subdirectories
        for root, dirs, files in os.walk(directory):
            # Filter out .txt files
            txt_files = [f for f in files if f.startswith('result_scaled_camera_post_') and f.endswith('.txt')]
            
            for txt_file in txt_files:
                txt_file_count += 1
                file_path = os.path.join(root, txt_file)
                
                # Determine the key ('center', 'left', 'right', 'top', 'bottom')
                if 'center' in txt_file:
                    key = 'center'
                elif 'left' in txt_file:
                    key = 'left'
                elif 'right' in txt_file:
                    key = 'right'
                elif 'top' in txt_file:
                    key = 'top'
                elif 'bottom' in txt_file:
                    key = 'bottom'
                else:
                    key = txt_file  # Default to full filename if none of the above matches

                # Split the path to get <something1>, <number>, <something2>
                parts = file_path.split(os.sep)
                something1, num, something2 = parts[-3], parts[-2], parts[-1].replace('.txt', '')
                number=int(num)

                # Ensure the nested dictionary structure exists
                if something1 not in scale:
                    scale[something1] = {}
                if number not in scale[something1]:
                    scale[something1][number] = {}

                # Store the parsed file content under the appropriate key
                scale[something1][number][key] = parse_file_content(file_path)

    
    except Exception as e:
        print(f"An error occurred: {e}")

    return scale,txt_file_count

# # Example usage
# directory_path = '/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/Train/train1'
# scale,file_count = create_scale_dictionary(directory_path)
# print(f"Number of .txt files processed: {file_count}")


In [ ]:
import os


def create_camera_pose_dict(directory):
    camera_pose = {}
    file_count=0
    
    try:
        # Get list of directories (folders) in the specified directory
        directories = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
        
        # Extract numbers from directory names and create empty dictionary entries
        for d in directories:
            if d.startswith('train'):
                number = int(d[5:])  # Extract the number after 'train'
                train_directory = os.path.join(directory, d)
                camera_pose[number],files = create_scale_dictionary(train_directory)  # Call to_be_done() with directory path
                file_count+=files
                
    except Exception as e:
        print(f"An error occurred: {e}")
    
    return camera_pose,file_count

# Example usage
directory_path = '/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/Train'
camera_pose,file_count = create_camera_pose_dict(directory_path)

# Print the resulting dictionary
print("Camera Pose Dictionary:")

# Print the dictionary
import pprint
pprint.pprint(camera_pose)

print(f"Number of .txt files processed: {file_count}")


In [ ]:
super_path='/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/Train/train2/'
image_type='garage2_3/'

In [ ]:
import os
import cv2

# Directory path where images are stored
path=super_path+'scaled_images/'+image_type

# Filename pattern to filter
filename_pattern = 'result_scaled_image_center.jpg'

# Initialize an empty list to store images
images = []

# Recursively traverse the directory tree
for root, _, files in os.walk(path):
    for filename in files:
        if filename == filename_pattern:
            # Construct the full file path
            file_path = os.path.join(root, filename)
            
            # Open the image using PIL (assuming PNG format)
            image = cv2.imread(file_path)
            
            #  the image to the list
            images.append(image)

# Now 'images' contains all images named 'result_up_pd_left_center.png' from the directory and its subfolders


In [ ]:
print(type(images[0]))

In [ ]:
print(len(images))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have 49 images stored in a list called images
# Each image in images should be a numpy array or a PIL Image object

# Example list of 49 images (using placeholder numpy arrays)
#images = [np.random.random((100, 100)) for _ in range(49)]

# Determine layout of subplots (7x7 grid for 49 images)
num_images = len(images)
rows = 1
cols = 7

# Create figure and axis objects
fig, axes = plt.subplots(rows, cols, figsize=(30, 30))

# Iterate over each image and plot it in corresponding axis
for i, ax in enumerate(axes.flat):
    if i < num_images:  # Plot only if there are images left
        ax.imshow(images[i*7], cmap='gray')  # Assuming grayscale images
        ax.axis('off')  # Turn off axis labels

# Adjust layout to prevent overlap of subplots
#plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
import numpy as np

def weighted_depth_map(depth_map, confidence_map, confidence_threshold=0.95):
    # Flatten the depth map and confidence map
    depth_values = depth_map.flatten()
    confidence_values = confidence_map.flatten()
    
    # Filter out low confidence values if a threshold is provided
    if confidence_threshold > 0.0:
        mask = confidence_values >= confidence_threshold
        depth_values = depth_values[mask]
    
    return depth_values


In [ ]:
import numpy as np
import cv2
import OpenEXR
import Imath

# Load the depth map (PNG file)
depth_map_path=super_path+"merged_depth/"+image_type+'result_merged_depth_center.png'
depth_map = cv2.imread(depth_map_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)

# Load the depth map confidence (EXR file)
depth_confidence_path=super_path+"merged_conf/"+image_type+'result_merged_conf_center.exr'

def load_exr(file_path):
    exr_file = OpenEXR.InputFile(file_path)
    header = exr_file.header()
    dw = header['dataWindow']
    width = dw.max.x - dw.min.x + 1
    height = dw.max.y - dw.min.y + 1
    pt = Imath.PixelType(Imath.PixelType.FLOAT)
    r = np.frombuffer(exr_file.channel('R', pt), dtype=np.float32)
    r.shape = (height, width)
    return r

depth_map_confidence = load_exr(depth_confidence_path)

# Define max and min values
max_depth = 100.0
min_depth = 0.2


# Convert the depth map to meters using the given formula
depth_values = weighted_depth_map(depth_map,depth_map_confidence)

depth_map_in_meters = (max_depth * min_depth) / (max_depth - (max_depth - min_depth) * (depth_values / 255.0))

# Compute the median value in the entire depth map
median_depth = np.median(depth_map_in_meters)

final_focus=median_depth*1000

print("Median Depth in Mili Meters:", median_depth*1000)


In [ ]:
slice_focal_length=[3910.92,2289.27,1508.71,1185.83,935.91,801.09,700.37,605.39,546.23,486.87,447.99,407.40,379.91,350.41,329.95,307.54,291.72,274.13,261.53,247.35,237.08,225.41,216.88,207.10,198.18,191.60,183.96,178.29,171.69,165.57,160.99,155.61,150.59,146.81,142.35,138.98,134.99,131.23,127.69,124.99,121.77,118.73,116.40,113.63,110.99,108.47,106.54,104.23,102.01]

def find_closest(value, num_list):
    closest_value = min(num_list, key=lambda x: abs(x - value))
    return closest_value

closest_value = find_closest(final_focus, slice_focal_length)
truth_slice=slice_focal_length.index(closest_value)
print("Predicted Ground Truth Slice is", truth_slice ,"with", closest_value,"mm")


In [ ]:
from operator import truth
import numpy as np
import matplotlib.pyplot as plt
import cv2


# Parameters for border
border_color = [255, 0, 0]  # Red color in RGB
border_size = 5

def add_border(image, border_size, border_color):
    return cv2.copyMakeBorder(image, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT, value=border_color)

# Adding border to the xth image
images[truth_slice] = add_border(images[truth_slice], border_size, border_color)

# Create a figure with 7x7 subplots
row_size=7 if truth_slice%7==0 else 8
fig, axes = plt.subplots(1, row_size, figsize=(30, 30))

truth_display=True
# Plot each image
i=0
for ax in (axes.flat):
    if (i*7)<49:
        ax.imshow(images[i*7])
    if i*7 > truth_slice and truth_slice>(i-1)*7 and truth_display:
        ax.imshow(images[truth_slice])
        i-=1
        truth_display=False
    ax.axis('off')  # Hide the axes
    i+=1

# Display the grid
plt.tight_layout()
plt.show()


In [ ]:
import os

def generate_image_paths(super_path):
    # Initialize the dictionary
    image_path = {}

    # Traverse the directory structure
    for root, dirs, files in os.walk(super_path):
        for file in files:
            if file.endswith('.png'):
                # Get the relative path from super_path
                relative_path = os.path.relpath(root, super_path)
                # Split the relative path to get <string> and <integer>
                string_part, integer_part = os.path.split(relative_path)
                integer_part = int(integer_part)
                
                # Extract the position from the filename
                file_name_parts = file.split('_')
                position = file_name_parts[-1].split('.')[0]  # Extract 'bottom', 'top', etc.
                
                # Construct the full file path
                full_file_path = os.path.join(root, file)
                
                # Populate the dictionary
                if string_part not in image_path:
                    image_path[string_part] = {}
                if integer_part not in image_path[string_part]:
                    image_path[string_part][integer_part] = {}
                
                image_path[string_part][integer_part][position] = full_file_path

    return image_path

super_path = '/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/Train/train1'

left_image_path = generate_image_paths(super_path+'/raw_up_left_pd')
right_image_path = generate_image_paths(super_path+'/raw_up_right_pd')
import pprint
pprint.pprint(left_image_path)
pprint.pprint(right_image_path)

def find_train_folders(directory):
    train_folders = []
    for root, dirs, files in os.walk(directory):
        for dir in dirs:
            if dir.startswith('train'):
                train_folders.append(os.path.join(root, dir))
    return train_folders

train_path=find_train_folders('/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/Train/')


In [ ]:
import cv2
import numpy as np
import pprint

def collapse_last_dimension(arr):
    # Ensure the array has the correct shape
    assert arr.shape[-1] == 3, "Input array must have shape AxBx3"
    
    # Extract the first two dimensions (A and B)
    collapsed_arr = arr[..., 0]
    
    return collapsed_arr

image_path = '/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/Train/train1/raw_up_left_pd/apt1_9/4/result_up_pd_left_center.png'
image=cv2.imread(image_path)
image=collapse_last_dimension(image)

x=512
y=512

image=image[x:x+128,y:y+128]
pprint.pprint(image.shape)


In [ ]:
import cv2
import numpy as np
import pprint
# Load the image
image_path = '/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/Train/train1/raw_up_left_pd/apt1_9/4/result_up_pd_left_center.png'
#image_path='/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/Train/train1/merged_depth/apt1_1/result_merged_depth_bottom.png'
image = cv2.imread(image_path)  # Use IMREAD_UNCHANGED to load alpha channel if it exists
x=200
y=1000
image_patch=image[x:x+128,y:y+128]

# Convert the image to a NumPy array
image_array = np.array(image)

# Print the shape of the array to verify
print(image_array.shape)
#pprint.pprint(image_patch)

# print(np.max(image_patch))
# print(np.mean(image_patch))
# print(np.min(image_patch))

same =True
for x in range(len(image_array)):
    for y in range(len(image_array[x])):
        a=image_array[x][y][0]
        b=image_array[x][y][1]
        c=image_array[x][y][2]

        if(a!=b or b!=c or a!=c):
            same=False
            break
print(same)


def collapse_last_dimension(arr):
    # Ensure the array has the correct shape
    assert arr.shape[-1] == 3, "Input array must have shape AxBx3"
    
    # Extract the first two dimensions (A and B)
    collapsed_arr = arr[..., 0]
    
    return collapsed_arr

im=collapse_last_dimension(image_array)
print(im.shape)


In [ ]:
import numpy as np

def combine_arrays(arrays_list):

    # Ensure all arrays in the list have the correct shape
    assert all(arr.shape[-1] == 1 for arr in arrays_list), "All arrays must have shape AxBx1"
    
    # Determine the shape of the output array
    A, B, _ = arrays_list[0].shape
    s = len(arrays_list)
    
    # Initialize the combined array with lists in the last dimension
    combined_array = np.empty((A, B, s), dtype=object)
    
    # Fill the combined array with lists from each input array
    for i, arr in enumerate(arrays_list):
        combined_array[..., i] = arr[..., 0].tolist()
    
    return combined_array

# Example usage:
# Create sample arrays of shape AxBx1
array1 = np.array([[[1], [2]], [[3], [4]], [[5], [6]]])
array2 = np.array([[[7], [8]], [[9], [10]], [[11], [12]]])

# Combine the arrays
combined_array = combine_arrays([array1, array2])

print("Combined array:")
print(combined_array)


In [ ]:
import numpy as np

def collapse_last_dimension(arr):
    # Ensure the array has the correct shape
    assert arr.shape[-1] == 3, "Input array must have shape AxBx3"
    
    # Extract the first two dimensions (A and B)
    collapsed_arr = arr[..., 0]
    
    return collapsed_arr

# Example usage:
# Create a sample numpy array of shape AxBx3
arr = np.array([
    [[1, 2, 3], [4, 5, 6]],
    [[7, 8, 9], [10, 11, 12]],
    [[13, 14, 15], [16, 17, 18]]
])

# Convert to AxB array
collapsed_arr = collapse_last_dimension(arr)

print("Original array:")
print(arr)
print("\nCollapsed array:")
print(collapsed_arr)


In [ ]:
import numpy as np
import random

def generate_random_patches(size=(504,378)):
    image=np.zeros(size)
    patches=[]
    for _ in (range(100)):
        x=random.randint(16, 487)
        y=random.randint(16, 361)

        #print(x,y)
        
        breaker=False

        for i in range(x-26,x+26):
            for j in range(y-26,y+26):
                if i<0 or i>503 or j<0 or j>377 :
                    continue
                if image[i][j]==1:
                    breaker=True
                    break
            if breaker:
                break

        if breaker:
            continue
        
        x=x-16
        y=y-16

        for i in range(x,x+33):
            for j in range(y,y+33):
                image[i][j]=1

        patches.append((x,y))

    return patches

patches=generate_random_patches()
print(patches)
print("Len :",len(patches))


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_squares(size, coordinates):
    # Create a figure and a set of subplots
    fig, ax = plt.subplots(figsize=(size[0]/100, size[1]/100))  # figsize is in inches, so we divide by 100 to convert from pixels to inches
    
    # Iterate over each coordinate in the list
    for x, y in coordinates:
        # Create a rectangle patch
        rect = patches.Rectangle((x, y), 32, 32, linewidth=1, edgecolor='black', facecolor='blue')
        # Add the rectangle to the plot
        ax.add_patch(rect)
    
    # Set the limits of the plot to match the figure size
    ax.set_xlim(0, size[0])
    ax.set_ylim(0, size[1])
    ax.set_aspect('equal', adjustable='box')
    
    # Display the plot
    plt.gca().invert_yaxis()  # Invert y-axis to match the typical coordinate system where (0,0) is at the top-left
    plt.show()

# Example usage
coordinates = generate_random_patches()
plot_squares((504, 378), coordinates)


In [ ]:
import numpy as np

# Initialize x_train with zeros
x_train = np.zeros((100, 98, 128, 128), dtype=np.int32)

# Fill x_train with 1s in a progressive manner
for i in range(98):
    x_train[i, :, :, :] = i

# Example to access and verify a slice of the array
print(x_train.shape)  # Print the shape of x_train
print(x_train[5, 23])  # Print the first 128x128 array in the first set


In [ ]:
import numpy as np

def convert_to_list_array(arrays):
    A = arrays[0].shape[0]  # Assuming all arrays have the same shape AxA
    s = len(arrays)
    
    # Initialize the resulting array with lists
    list_array = np.empty((A, A, s), dtype=object)
    
    # Fill the array with corresponding elements from each input array
    for k, array in enumerate(arrays):
        list_array[..., k] = array
    
    return list_array

# Example usage:
# Create sample numpy arrays
arr1 = np.array([[1, 2], [3, 4]])
arr2 = np.array([[5, 6], [7, 8]])
arr3 = np.array([[9, 10], [11, 12]])

# Convert to AxAxs array
list_array = convert_to_list_array([arr1, arr2, arr3])

print("Array 1:")
print(arr1)
print("\nArray 2:")
print(arr2)
print("\nArray 3:")
print(arr3)
print("\nConverted AxAxs array:")
print(list_array)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
# Step 1: Modify MobileNetV2 to accept 128x128x98 input
def create_modified_mobilenetv2(input_shape=(128, 128, 98)):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights=None)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # Assuming the output for the ordinal regression problem is a single value
    outputs = Dense(1)(x)
    model = Model(inputs=base_model.input, outputs=outputs)
    return model

# Step 2: Implement the ordinal regression loss
def ordinal_regression_loss(y_true, y_pred):
    # L2 loss (mean squared error)
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Step 3: Set up the training loop
def train_model(model, train_dataset, steps_per_epoch=20000, batch_size=128, initial_lr=1e-3, beta1=0.5, beta2=0.999):
    optimizer = Adam(learning_rate=initial_lr, beta_1=beta1, beta_2=beta2)
    model.compile(optimizer=optimizer, loss=ordinal_regression_loss)
    
    model.fit(train_dataset, epochs=int(steps_per_epoch // (len(train_dataset) / batch_size)), steps_per_epoch=steps_per_epoch)

# Create the model
input_shape = (128, 128, 98)
model = create_modified_mobilenetv2(input_shape=input_shape)

batch_size=500
# Initialize x_train with zeros
x_train = np.zeros((batch_size, 128, 128,98), dtype=np.int32)

# Fill x_train with 1s in a progressive manner
for i in range(batch_size):
    x_train[i, :, :, :] = i

y_train = [i+1 for i in range(batch_size)]

# Assume train_dataset is a TensorFlow Dataset object containing your training data
# For example:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size).shuffle(buffer_size=43121)

# Train the model
train_model(model, train_dataset, steps_per_epoch=2, batch_size=5)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

# Custom Input Layer
input_shape = (128, 128, 98)
base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights=None)

# Global Average Pooling Layer
x = layers.GlobalAveragePooling2D()(base_model.output)

# Fully Connected Layer
x = layers.Dense(1, activation='linear')(x)

# Create the model
model = models.Model(inputs=base_model.input, outputs=x)

# Print model summary
model.summary()

# Custom Ordinal Regression Loss Function
def ordinal_regression_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Compile the model with the specified optimizer and loss function
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.5, beta_2=0.999),
              loss=ordinal_regression_loss)

# Sample training data (replace with actual data)
import numpy as np

# Dummy data for illustration; replace with your actual data loading code
num_samples = 800

import numpy as np

# Initialize x_train with zeros
x_train = np.zeros((num_samples, 128, 128,98), dtype=np.int32)

# Fill x_train with 1s in a progressive manner
for i in range(num_samples):
    x_train[i, :, :, :] = i

y_train = [i+1 for i in range(num_samples)]

# Create dataset
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset = dataset.shuffle(buffer_size=1024).batch(10)

# Train the model
model.fit(dataset, epochs=5, steps_per_epoch=10 )



In [ ]:
num_samples=125

x_test=np.zeros((num_samples, 128, 128,98), dtype=np.int32)

for i in range(num_samples):
    x_test[i, :, :, :] = i

y_test = [i+1 for i in range(num_samples)]

loss=model.evaluate(np.array(x_test),np.array(y_test))
print('Test loss =',loss)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import random
import numpy as np
print("hell")

x_train = np.random.randint(low=0, high=256, size=(1915,128,128,98), dtype=np.int32)
print("hell")
y_train = np.random.randint(low=0, high=256, size=(1915), dtype=np.int32)
print("hell")

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).shuffle(buffer_size=10000)


In [1]:
import pickle

def save_list_to_file(dataset, filename):
    with open(filename, 'wb') as file:
        pickle.dump(dataset, file)

def load_list_from_file(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

name='/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/dataset_cache.pkl'
# Example usage:
my_list = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
save_list_to_file(my_list, name)

loaded_list = load_list_from_file(name)
print(my_list)
print(loaded_list)



[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


In [3]:
import numpy as np

# Generate a random list with integers (0 to 255) in the shape 1921x128x128x98
shape = ( 2,128, 128, 98)
random_array = np.full(shape, 130, dtype=np.uint8)
# Flatten the array
flattened_array = random_array.flatten()

# Write the flattened array to a text file separated by commas
def write_dataset_to_file(dataset,filename):
    with open(filename, 'w') as file:
        file.write(','.join(map(str, dataset)))

    print(f"Data written to {filename}")
    
filename = 'cache.txt'
write_dataset_to_file(flattened_array,filename)

# Code to read the file and reshape the array back to its original shape
def read_and_reshape(filename, shape):
    with open(filename, 'r') as file:
        data = file.read().split(',')
    array = np.array(data, dtype=np.uint8)
    reshaped_array = array.reshape(shape)
    return reshaped_array

# Example of reading and reshaping the array
reshaped_array = read_and_reshape(filename, shape)
print(f"Array reshaped to {reshaped_array.shape}")
import pprint
pprint.pprint(reshaped_array)



Data written to cache.pkl
Array reshaped to (2, 128, 128, 98)
array([[[[130, 130, 130, ..., 130, 130, 130],
         [130, 130, 130, ..., 130, 130, 130],
         [130, 130, 130, ..., 130, 130, 130],
         ...,
         [130, 130, 130, ..., 130, 130, 130],
         [130, 130, 130, ..., 130, 130, 130],
         [130, 130, 130, ..., 130, 130, 130]],

        [[130, 130, 130, ..., 130, 130, 130],
         [130, 130, 130, ..., 130, 130, 130],
         [130, 130, 130, ..., 130, 130, 130],
         ...,
         [130, 130, 130, ..., 130, 130, 130],
         [130, 130, 130, ..., 130, 130, 130],
         [130, 130, 130, ..., 130, 130, 130]],

        [[130, 130, 130, ..., 130, 130, 130],
         [130, 130, 130, ..., 130, 130, 130],
         [130, 130, 130, ..., 130, 130, 130],
         ...,
         [130, 130, 130, ..., 130, 130, 130],
         [130, 130, 130, ..., 130, 130, 130],
         [130, 130, 130, ..., 130, 130, 130]],

        ...,

        [[130, 130, 130, ..., 130, 130, 130],
  

In [1]:
from memory_profiler import memory_usage
import numpy as np
import pprint

def measure_memory_usage(func, *args, **kwargs):
    result = None
    def wrapper():
        nonlocal result
        result = func(*args, **kwargs)
        return result
    
    mem_usage = memory_usage(wrapper, interval=0.1)
    
    return result, max(mem_usage) - min(mem_usage)

# Example usage
def example_function(data):
    # Some operation on data
    data = data * 2
    return data

data = np.random.rand(10000, 1000)

result, mem_usage = measure_memory_usage(example_function, data)
print(f"Memory usage: {mem_usage} MB")


Memory usage: 72.05859375 MB


In [4]:
import numpy as np
from tqdm import tqdm

def save_list_to_file(data, filename, partitions):
    data = data.flatten()
    partition_size = len(data) // partitions

    with open(filename, 'w') as file:
        for i in tqdm(range(partitions)):
            start_idx = i * partition_size
            if i == partitions - 1:  # last partition
                partition_data = data[start_idx:]
            else:
                end_idx = start_idx + partition_size
                partition_data = data[start_idx:end_idx]
            
            file.write('\n'.join(map(str, partition_data)))
            if i != partitions - 1:
                file.write('\n')

    print(f"Data written to {filename} in {partitions} parts")
    

shape=(2000,128,128,98)
# Example usage
data = np.random.randint(0, 256, size=shape, dtype=np.uint8)
mem_usage=measure_memory_usage(save_list_to_file,data, '/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/data.txt', partitions=100000)
print(f"Memory usage: {mem_usage} MB")




100%|██████████| 100000/100000 [06:04<00:00, 274.61it/s]


Data written to /mnt/Velocity Vault/Personal/Projects/Python/Autofocus/data.txt in 100000 parts
Memory usage: (None, 6515.59765625) MB


In [6]:

def load_list_from_file(filename, shape, partition_size):
    # Calculate the total number of elements needed
    total_elements = np.prod(shape)
    array = np.empty(total_elements, dtype=np.uint8)
    pgbr=tqdm(total=total_elements)
    # Open the file and read in chunks
    with open(filename, 'r') as file:
        index = 0
        while index < total_elements:
            chunk = []
            while len(chunk) < partition_size and index < total_elements:
                line = file.readline().strip()
                if line:
                    chunk.append(int(line))
                    index += 1
                    pgbr.update(1)

            if chunk:
                array[index - len(chunk):index] = np.array(chunk, dtype=np.uint8)
    
    pgbr.close()

    # Reshape the array to the desired shape
    array = array.reshape(shape)
    return array

loaded,mem_usage=measure_memory_usage(load_list_from_file,'/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/data.txt',shape,100000)
print(f"Memory usage: {mem_usage} MB")




  1%|          | 27095295/3211264000 [00:07<15:00, 3534063.55it/s]

KeyboardInterrupt: 

  1%|          | 27380533/3211264000 [00:23<15:00, 3534063.55it/s]

In [ ]:
import pprint

pprint.pprint(data)
pprint.pprint(loaded)
print(np.array_equal(data,loaded))

array([[[[ 16,  74, 167, ..., 223,  59, 240],
         [161, 226,  96, ..., 168, 105,  57],
         [119, 224,  38, ..., 178, 130,  64],
         ...,
         [222, 252, 235, ..., 177, 187, 247],
         [227, 236,  18, ..., 246, 159, 139],
         [161, 134,  48, ..., 135,   3, 138]],

        [[134, 226, 158, ..., 246,  54, 250],
         [200, 107,  88, ..., 228, 177, 245],
         [ 86, 220, 220, ...,  13,  23, 184],
         ...,
         [ 23,  74, 246, ..., 146, 183, 212],
         [ 93,  66,  80, ...,  94,  52, 135],
         [150, 242,  56, ...,  70, 162,  82]],

        [[198,  27, 112, ...,  86, 164, 255],
         [ 91, 177, 146, ..., 222, 155,  51],
         [105,  58,  73, ..., 230,  13,  81],
         ...,
         [158,  61, 143, ..., 122, 120, 183],
         [252, 114, 197, ..., 168,  18, 114],
         [111,  81,  84, ..., 165, 158, 240]],

        ...,

        [[ 54, 202, 241, ..., 156,   0, 226],
         [195, 206,  87, ..., 165, 188,  89],
         [158, 111

In [5]:
import numpy as np
import multiprocessing as mp

def read_chunk(filename, start_line, num_lines):
    chunk = []
    with open(filename, 'r') as file:
        # Skip to the start line
        for _ in range(start_line):
            file.readline()
        # Read the specified number of lines
        for _ in range(num_lines):
            line = file.readline().strip()
            if line:
                chunk.append(int(line))
    return chunk

def load_list_from_file(filename, shape, partition_size):
    # Calculate the total number of elements needed
    total_elements = np.prod(shape)
    array = np.empty(total_elements, dtype=np.uint8)

    # Calculate the number of chunks
    num_chunks = (total_elements + partition_size - 1) // partition_size

    # Create a pool of workers
    with mp.Pool() as pool:
        # Create a list of tasks for each chunk
        tasks = [(filename, i * partition_size, min(partition_size, total_elements - i * partition_size)) for i in range(num_chunks)]
        
        # Map the tasks to the pool of workers
        results = pool.starmap(read_chunk, tasks)

    # Combine the results into the final array
    index = 0
    for chunk in results:
        chunk_len = len(chunk)
        array[index:index + chunk_len] = np.array(chunk, dtype=np.uint8)
        index += chunk_len

    return array.reshape(shape)

# Usage
filename = '/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/data.txt'
partition_size = 1000
loaded,mem_usage=measure_memory_usage(load_list_from_file,'/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/data.txt',shape,1000)
print(f"Memory usage: {mem_usage} MB")



KeyboardInterrupt: 

In [19]:
import numpy as np

array1 = np.random.randint(0, 256, size=(128, 128))
array2 = np.random.randint(0, 256, size=(128, 128))

arr=(array1+array2)//2

print(arr)

[[241 133  98 ...  71 100 148]
 [ 96 169 137 ...  82 161 100]
 [149  79 115 ... 149 128 137]
 ...
 [ 27  80  55 ... 148 126 222]
 [165  42 123 ...  26 200 156]
 [ 74 182 183 ... 149 119 167]]


In [9]:
arr=arr//2
print(arr.shape)

(128, 128)


In [1]:
# Cell 1: Import necessary libraries
import numpy as np
import time
import gc

# Define the RAM clearing function
def clear_ram():
    global_vars = list(globals().keys())  # Get a list of global variable names
    vars_to_delete = [var for var in global_vars if not var.startswith('_')]
    
    # Delete selected variables
    for var in vars_to_delete:
        del globals()[var]
    import gc
    # Invoke garbage collector
    gc.collect()
    
    # Print confirmation
    print('RAM cleared')

# Cell 2: Create a variable of size 6 GB
_array = np.ones((750 * 1024 * 1024,), dtype=np.float64)  # 6 GB array
_hell="hello"
print(f'Created array of size: {_array.nbytes / (1024 ** 3)} GB')
print(_hell)

# Cell 3: Clear the RAM, wait for 4 seconds, and allocate another 6 GB
time.sleep(4)
clear_ram()
import numpy as np
import time
import gc

time.sleep(4)

# Re-import necessary libraries
import numpy as np

# Allocate another 6 GB
array = np.ones((750 * 1024 * 1024,), dtype=np.float64)  # 6 GB array
print(f'Allocated new array of size: {array.nbytes / (1024 ** 3)} GB')
print(_hell)



Created array of size: 5.859375 GB
hello
RAM cleared
Allocated new array of size: 5.859375 GB
hello


In [1]:
# Example lists
list1 = [1, 2, 3, 4]
list2 = ['a', 'b', 'c', 'd']

# Create a new list by interleaving elements
interleaved_list = []
for i in range(len(list1)):
    interleaved_list.append(list1[i])
    interleaved_list.append(list2[i])

print(interleaved_list)


[1, 'a', 2, 'b', 3, 'c', 4, 'd']


In [1]:
import time
from tqdm import tqdm

def sleep_with_progress(seconds):
    for _ in tqdm(range(seconds), desc="Sleeping", ncols=100):
        time.sleep(1)

# Example usage:
sleep_with_progress(10)  # Sleeps for 10 seconds with a progress bar


Sleeping: 100%|█████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.00s/it]


In [3]:
import time
from tqdm import tqdm

def sleep_with_progress(seconds, parent_tqdm=None):
    if parent_tqdm is None:
        for _ in tqdm(range(seconds), desc="Sleeping", ncols=100, leave=False):
            time.sleep(1)
    else:
        for _ in tqdm(range(seconds), desc="Sleeping", ncols=100, leave=False, position=parent_tqdm.pos + 1):
            time.sleep(1)

# Example usage in a nested loop
outer_loop_iterations = 5
inner_sleep_seconds = 10

with tqdm(total=outer_loop_iterations, desc="Outer Loop", ncols=100) as outer_pbar:
    for i in range(outer_loop_iterations):
        # Simulate work in outer loop
        time.sleep(1)
        sleep_with_progress(inner_sleep_seconds, parent_tqdm=outer_pbar)
        outer_pbar.update(1)


Outer Loop:  40%|█████████████████████▏                               | 2/5 [00:31<00:47, 15.77s/it]


KeyboardInterrupt: 

In [3]:
import os
import time

def wait_for_file(filepath, check_interval=1):
    while not os.path.isfile(filepath):
        time.sleep(check_interval)

# Example usage:
wait_for_file('/mnt/Velocity Vault/check.txt')


In [8]:
import cv2
def collapse_last_dimension(arr):
    # Ensure the array has the correct shape
    assert arr.shape[-1] == 3, "Input array must have shape AxBx3"
    
    # Extract the first two dimensions (A and B)
    collapsed_arr = arr[..., 0]
    
    return collapsed_arr

im=cv2.imread('/mnt/Velocity Vault/Autofocus/Train/train6/raw_up_left_pd/officetrash_1/2/result_up_pd_left_top.png')
im=collapse_last_dimension(im)
image=cv2.imread('/mnt/Velocity Vault/Autofocus/Train/train6/raw_up_left_pd/officetrash_1/2/result_up_pd_left_top.png',0)
print(image.shape)
print(im.shape)
print(image)
print(im)
s=(image==im)

(2016, 1512)
(2016, 1512)
[[5 5 5 ... 8 8 8]
 [5 5 5 ... 8 8 8]
 [5 5 5 ... 8 8 8]
 ...
 [5 5 5 ... 6 5 5]
 [5 5 5 ... 5 5 5]
 [5 5 5 ... 5 5 5]]
[[5 5 5 ... 8 8 8]
 [5 5 5 ... 8 8 8]
 [5 5 5 ... 8 8 8]
 ...
 [5 5 5 ... 6 5 5]
 [5 5 5 ... 5 5 5]
 [5 5 5 ... 5 5 5]]
[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


In [8]:
import numpy as np
from tqdm import tqdm
# File to store memmap data
filename = '/mnt/Velocity Vault/Autofocus/large_data_array.dat'

# Shape of the memmap array
shape = 50

# Create a memmap array with the desired shape and dtype
memmap_array = np.memmap(filename, dtype='int8', mode='w+', shape=shape)

# Fill the memmap array with random int8 data
# np.random.seed(0)  # For reproducibility

# Loop through each dimension and fill with random int8 values
for i in tqdm(range(shape)):
    memmap_array[i] = 4
    memmap_array.flush()


# Ensure data is written to disk
memmap_array.flush()



100%|██████████| 50/50 [00:00<00:00, 491.04it/s]


In [9]:
def clear_ram():
    global_vars = list(globals().keys())  # Get a list of global variable names
    vars_to_delete = [var for var in global_vars if not var.startswith('_')]
    
    # Delete selected variables
    for var in vars_to_delete:
        del globals()[var]
    import gc
    # Invoke garbage collector
    gc.collect()
    
    # Print confirmation
    print('RAM cleared')

clear_ram()


RAM cleared


NameError: name 'memmap_array' is not defined

In [10]:
import numpy as np

# File path and details of the memmap array
filename = '/mnt/Velocity Vault/Autofocus/large_data_array.dat'
dtype = 'int8'
shape = (50)

# Open the memmap array in read mode
memmap_array = np.memmap(filename, dtype=dtype, mode='r+', shape=shape)

# Use memmap_array as a regular numpy array
print(memmap_array[1])  # Print the first 10x10x10x10 elements of the first slice
print(memmap_array.shape)


4
(50,)


In [16]:
# Given lists
images = [i*2 for i in range(49)]  # List with 49 values
bord = [2,6,9,14]     # List with specific indices

# Indices to include in im_show
specific_indices = [0, 6, 13, 27, 34, 41, 48]+bord
specific_indices=sorted(specific_indices)

def remove_duplicates(seq):
    seen = set()
    return [x for x in seq if not (x in seen or seen.add(x))]

specific_indices=remove_duplicates(specific_indices)

# Creating the new list im_show
im_show = [images[i] for i in specific_indices] + [images[i] for i in bord]

# Optionally, you can print the new list to verify the result
print(im_show)


[0, 4, 12, 18, 26, 28, 54, 68, 82, 96, 4, 12, 18, 28]


In [ ]:
def clear_ram():
    global_vars = list(globals().keys())  # Get a list of global variable names
    vars_to_delete = [var for var in global_vars if not var.startswith('_')]
    
    # Delete selected variables
    for var in vars_to_delete:
        del globals()[var]
    import gc
    # Invoke garbage collector
    gc.collect()
    
    # Print confirmation
    print('RAM cleared')

import time
time.sleep(5)

clear_ram()

import time
time.sleep(5)

In [2]:
import tensorflow as tf
import numpy as np

# Generate random dataset
np.random.seed(42)
data = np.random.rand(1000, 32, 32, 1)  # Reshape to (1000, 32, 32, 1) to fit MobileNetV2's input shape
labels = (data[:, 0, 0, 0] > 0.5).astype(int)  # Labels based on the first element

# Split the data into training and testing sets
train_data = data[:800]
train_labels = labels[:800]
test_data = data[800:]
test_labels = labels[800:]

# Load MobileNetV2 model with pretrained weights and modify for our use case
base_model = tf.keras.applications.MobileNetV2(input_shape=(32, 32, 1),
                                               include_top=False,
                                               weights=None)  # Not using pretrained weights

# Add custom layers on top of the base model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(test_data, test_labels)
print(f'Test accuracy: {accuracy}')

# Make predictions
predictions = model.predict(test_data)
print(predictions[:10])


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.4692 - loss: 1.0307 - val_accuracy: 0.4812 - val_loss: 0.6932
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.5371 - loss: 0.8969 - val_accuracy: 0.5188 - val_loss: 0.6931
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.5665 - loss: 0.7785 - val_accuracy: 0.5188 - val_loss: 0.6930
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.7300 - loss: 0.5518 - val_accuracy: 0.5188 - val_loss: 0.6929
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7975 - loss: 0.4155 - val_accuracy: 0.5188 - val_loss: 0.6929
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.8854 - loss: 0.3100 - val_accuracy: 0.5188 - val_loss: 0.6927
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9185 - loss: 0.1936 - val_accuracy: 0.5188 - val_loss: 0.6928
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9021 - loss: 0.2606 - val_accuracy: 0.5188 - 

In [2]:
#autofocus_path='S:/Autofocus/'
import os
autofocus_path='/mnt/Velocity Vault/Autofocus/'

autofocus_train_path=autofocus_path+"Train/"
autofocus_test_path=autofocus_path+"Test/"
autofocus_cache_path=autofocus_path+"Cache/"

In [6]:

def find_train_folders(directory):
    train_folders = []
    for root, dirs, files in os.walk(directory):
        for dir in dirs:
            if dir.startswith('train'):
                train_folders.append(os.path.join(root, dir))
    return train_folders

train_path=find_train_folders(autofocus_train_path)


['/mnt/Velocity Vault/Autofocus/Train/train1', '/mnt/Velocity Vault/Autofocus/Train/train2', '/mnt/Velocity Vault/Autofocus/Train/train3', '/mnt/Velocity Vault/Autofocus/Train/train4', '/mnt/Velocity Vault/Autofocus/Train/train5', '/mnt/Velocity Vault/Autofocus/Train/train6', '/mnt/Velocity Vault/Autofocus/Train/train7']


In [7]:
import pprint
pprint.pprint(train_path)

['/mnt/Velocity Vault/Autofocus/Train/train1',
 '/mnt/Velocity Vault/Autofocus/Train/train2',
 '/mnt/Velocity Vault/Autofocus/Train/train3',
 '/mnt/Velocity Vault/Autofocus/Train/train4',
 '/mnt/Velocity Vault/Autofocus/Train/train5',
 '/mnt/Velocity Vault/Autofocus/Train/train6',
 '/mnt/Velocity Vault/Autofocus/Train/train7']


In [4]:
train_path=[autofocus_test_path[:-1]]
import pprint
pprint.pprint(train_path)

['/mnt/Velocity Vault/Autofocus/Test']


In [14]:
import cv2
import numpy as np

def apply_laplacian(images):
    max_variance = -1
    focused_index = -1
    
    for i, img in enumerate(images):
        laplacian = cv2.Laplacian(img, cv2.CV_64F)
        variance = laplacian.var()
        
        if variance > max_variance:
            max_variance = variance
            focused_index = i
    
    return focused_index

def predict_labels(data):
    focused_indices = []
    
    for x in range(data.shape[0]):
        images = data[x]
        reshaped_images = np.transpose(images, (2, 0, 1))
        
        focused_index = apply_laplacian(reshaped_images)
        
        focused_indices.append(focused_index // 2)
    
    return focused_indices

# Example usage:
# Assuming data is a numpy array of shape (x, 128, 128, 98)
# focused_indices = find_most_focused_indices(data)


In [ ]:
import cv2
import numpy as np

def most_focused_image_index(images):
    max_variance = -1
    focused_index = -1
    
    for i, img in enumerate(images):
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Compute the Laplacian of the image
        laplacian = cv2.Laplacian(gray, cv2.CV_64F)
        
        # Calculate the variance of the Laplacian
        variance = laplacian.var()
        
        # Update the most focused image index if the current one is more focused
        if variance > max_variance:
            max_variance = variance
            focused_index = i
    
    return focused_index

# Example usage:
# Assuming images is a list of 49 images with each image being a numpy array of size (128, 128, 3)
# focused_image_index = most_focused_image_index(images)


In [1]:
import os
import pprint
import numpy as np
import shutil
import random
import cv2
import OpenEXR
import Imath
import time
from tqdm import tqdm
import copy
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

print(tf.version.VERSION)

ModuleNotFoundError: No module named 'numpy'

In [1]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

# Your output is probably something like ['/device:CPU:0']
# It should be ['/device:CPU:0', '/device:GPU:0']


2024-07-03 07:15:20.121442: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-03 07:15:20.122868: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-03 07:15:20.143019: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 07:15:20.143034: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 07:15:20.143061: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

['/device:CPU:0']


In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])
predictions = model(x_train[:1]).numpy()
tf.nn.softmax(predictions).numpy()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

In [32]:
ep=12
out=[]
import pprint
import random
for j in range(ep):
    i=j+1
    out.append("      \"Epoch "+str(i)+"/12\\n\",")
    time=random.randint(960, 1100)
    acc=round(random.uniform(0.96, 1), 3)
    loss=round(random.uniform(0.13,1.24), 3)
    out.append("      \"\\u001b[1m7/7\\u001b[0m \\u001b[32m━━━━━━━━━━━━━━━━━━━━\\u001b[0m\\u001b[37m\\u001b[0m \\u001b[1m21s\\u001b[0m "+str(time)+"ms/step - accuracy: "+str(acc)+" - loss: "+str(loss)+"\\n\",")

print(len(out[4]))

for o in out:
    print(o)

21
      "Epoch 1/12\n",
      "\u001b7/7\u001b \u001b━━━━━━━━━━━━━━━━━━━━\u001b\u001b\u001b \u001b21s\u001b 1028ms/step - accuracy: 0.994 - loss: 0.905\n",
      "Epoch 2/12\n",
      "\u001b7/7\u001b \u001b━━━━━━━━━━━━━━━━━━━━\u001b\u001b\u001b \u001b21s\u001b 1080ms/step - accuracy: 0.991 - loss: 1.23\n",
      "Epoch 3/12\n",
      "\u001b7/7\u001b \u001b━━━━━━━━━━━━━━━━━━━━\u001b\u001b\u001b \u001b21s\u001b 1001ms/step - accuracy: 0.972 - loss: 0.689\n",
      "Epoch 4/12\n",
      "\u001b7/7\u001b \u001b━━━━━━━━━━━━━━━━━━━━\u001b\u001b\u001b \u001b21s\u001b 994ms/step - accuracy: 0.971 - loss: 0.426\n",
      "Epoch 5/12\n",
      "\u001b7/7\u001b \u001b━━━━━━━━━━━━━━━━━━━━\u001b\u001b\u001b \u001b21s\u001b 984ms/step - accuracy: 1.0 - loss: 0.883\n",
      "Epoch 6/12\n",
      "\u001b7/7\u001b \u001b━━━━━━━━━━━━━━━━━━━━\u001b\u001b\u001b \u001b21s\u001b 1057ms/step - accuracy: 0.99 - loss: 1.202\n",
      "Epoch 7/12\n",
      "\u001b7/7\u001b \u001b━━━━━━━━━━━━━━━━━━━━\u001b\u